In [1]:
#-----  librerias  ----- #
import numpy as np
import pandas as pd
import pandapower as pp
from pandapower.pf.runpp_3ph import runpp_3ph
from pandapower.plotting.plotly import simple_plotly
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from pandapower.plotting.plotly import pf_res_plotly
import networkx as nx
from rtree import index
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta, time
import pandapower.networks as pn

# <img src="Data/Imagenes/Cigre_MV_LV.png" alt="comparacion" width="50%">

# Costos duales

## Prueba 1
    - Cargas fijas
    - Sin restricciones
    - Un costo de 100$/1MW para la cabecera del alimentador

In [2]:
# Definicion de la red
net = pn.create_cigre_network_lv()

net.switch.loc[1, 'closed'] = False
net.switch.loc[2, 'closed'] = False
net.switch.loc[0, 'closed'] = True  #Solo switch residencial activado

loads = [0, 1, 2, 3, 4, 5] #Cargas residenciales
no_loads = [ 6, 7, 8, 9, 10, 11, 12, 13, 14] #Cargas no residenciales

net.load.loc[loads, 'q_mvar'] = 0
net.load.loc[no_loads, 'in_service'] = False
net.load.loc[net.load['name'] == 'Load R1', 'p_mw'] *= 1.7

##Restricciones
for bus in net.bus.index:
    net.bus.at[bus, 'min_vm_pu'] = 0.8
    net.bus.at[bus, 'max_vm_pu'] = 1.2

for trafo in net.trafo.index:
    net.trafo.at[trafo, 'max_loading_percent'] = 200

for line in net.line.index:
    net.line.at[line, 'max_loading_percent'] = 200  # Considerar que la potencia nominal del transformador son 0.5 MVA


## Costos para la función objetivo
pp.create_pwl_cost(net, 0, 'ext_grid', [[0, 1.0, 100]])  # Un costo de 100$ para 1 MW


0

In [3]:
# Perfil de consumo de las cargas

Filename_historico = 'Data/Input/GAL.csv'  
Demanda_alimentadores = pd.read_csv(Filename_historico,delimiter=',')
Demanda_alimentadores.head()

Demanda_alimentadores['timestamp'] = pd.to_datetime(Demanda_alimentadores['timestamp'])

fig_load = go.Figure()

load_columns = [col for col in Demanda_alimentadores.columns if col.startswith('load')]

for col in load_columns:
    fig_load.add_trace(go.Scatter(
        x=Demanda_alimentadores['timestamp'], 
        y=Demanda_alimentadores[col],
        mode='lines', 
        name=col
    ))

fig_load.add_shape(
    type='line', 
    x0=Demanda_alimentadores['timestamp'].min(), 
    x1=Demanda_alimentadores['timestamp'].max(), 
    y0=89, 
    y1=89, 
    line=dict(color='red', width=2)
)

fig_load.update_layout(
    title="Perfiles de carga de todos los alimentadores",
    xaxis_title="Tiempo",
    yaxis_title="Carga (kW)",
    template="plotly_white",
    legend_title="Alimentadores"
)

fig_load.update_layout(
    legend=dict(
        orientation="v",
        yanchor="middle",
        y=0.5,
        xanchor="left",
        x=1.02,
        itemsizing="constant"
    ),
    margin=dict(r=150)  
)

fig_load.show()

In [4]:
df_res_bus = pd.DataFrame()
df_res_line = pd.DataFrame()
df_res_trafo = pd.DataFrame()
df_res_ext_grid = pd.DataFrame()
df_res_load = pd.DataFrame()
df_res_shunt = pd.DataFrame()
df_res_sgen = pd.DataFrame()
df_res_gen = pd.DataFrame()
df_cantidad_elementos = pd.DataFrame()

df_res_cost = pd.DataFrame()
df_cargas_restringidas = pd.DataFrame(columns=['timestamp', 'load_name', 'expected_p_mw', 'real_p_mw', 'restricted_p_mw'])

frames = []

for index, row in Demanda_alimentadores.iterrows():
    fecha_local = row['timestamp']
    
    for idx, load_row in net.load.iterrows():
        load_name = load_row['name']
        
        if load_name == 'Load R11':
            expected_p_mw = float(row['load1']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 4000
        
        elif load_name == 'Load R15':
            expected_p_mw = float(row['load10']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 15000
        
        elif load_name == 'Load R16':
            expected_p_mw = float(row['load15']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 5500
        
        elif load_name == 'Load R17':
            expected_p_mw = float(row['load20']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 21000
        
        elif load_name == 'Load R18':
            expected_p_mw = float(row['load25']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 21000

    pp.runopp(net,verbose=False) 
    
    net.res_bus['datetime'] = fecha_local
    net.res_line['datetime'] = fecha_local
    net.res_trafo['datetime'] = fecha_local
    net.res_ext_grid['datetime'] = fecha_local
    net.res_load['datetime'] = fecha_local
    net.res_shunt['datetime'] = fecha_local
    net.res_sgen['datetime'] = fecha_local
    net.res_gen['datetime'] = fecha_local

    df_res_cost_xd = pd.DataFrame({'Cost': [net.res_cost], 'datetime': [fecha_local]})

    net.res_gen['type'] = net.gen['type']

    net.res_bus['bus_name'] = net.bus['name']
    net.res_line['line_name'] = net.line['name']
    net.res_trafo['trafo_name'] = net.trafo['name']
    net.res_ext_grid['ext_grid_name'] = net.ext_grid['name']
    net.res_load['load_name'] = net.load['name']
    net.res_shunt['shunt_name'] = net.shunt['name']
    net.res_sgen['sgen_name'] = net.sgen['name']
    net.res_gen['gen_name'] = net.gen['name']

    net.res_sgen['bus'] = net.sgen['bus']
    net.res_gen['bus'] = net.gen['bus']
    net.res_load['type'] = net.load['type']

    ext_grid_buses = net.ext_grid['bus'].values
    ext_grid_bus_names = net.bus.loc[ext_grid_buses, 'name'].values

    net.res_ext_grid['bus_name'] = ext_grid_bus_names
    net.res_ext_grid['bus_index'] = ext_grid_buses

    load_buses = net.load['bus'].values
    load_bus_names = net.bus.loc[load_buses, 'name'].values
    net.res_load['bus_name'] = load_bus_names
    net.res_load['bus_index'] = load_buses
    from_bus_names = net.bus.loc[net.line['from_bus'], 'name'].values
    to_bus_names = net.bus.loc[net.line['to_bus'], 'name'].values

    net.res_line['from_bus_name'] = from_bus_names
    net.res_line['to_bus_name'] = to_bus_names
    net.res_line['from_bus'] = net.line['from_bus']
    net.res_line['to_bus'] = net.line['to_bus']

    line_active_losses = net.res_line.pl_mw.sum()
    line_reactive_losses = net.res_line.ql_mvar.sum()

    df_temp = pd.DataFrame({
        'Perdidas de linea potencia activa (MW)': [line_active_losses],
        'Perdidas de linea potencia reactiva (MVAR)': [line_reactive_losses],
        'datetime': [fecha_local]
    })
    df_res_cost = pd.concat([df_res_cost, df_res_cost_xd])
    df_res_bus = pd.concat([df_res_bus, net.res_bus])
    df_res_line = pd.concat([df_res_line, net.res_line])
    df_res_trafo = pd.concat([df_res_trafo, net.res_trafo])
    df_res_ext_grid = pd.concat([df_res_ext_grid, net.res_ext_grid])
    df_res_load = pd.concat([df_res_load, net.res_load])
    df_res_shunt = pd.concat([df_res_shunt, net.res_shunt])
    df_res_sgen = pd.concat([df_res_sgen, net.res_sgen])
    df_res_gen = pd.concat([df_res_gen, net.res_gen])
    df_cantidad_elementos = pd.concat([df_cantidad_elementos, df_temp])

with pd.ExcelWriter('Data/Output/Results.xlsx') as writer:
    df_res_cost.to_excel(writer, sheet_name='Cost')
    df_res_bus.to_excel(writer, sheet_name='Bus Results')
    df_res_line.to_excel(writer, sheet_name='Line Results')
    df_res_trafo.to_excel(writer, sheet_name='Transformer Results')
    df_res_ext_grid.to_excel(writer, sheet_name='External Grid Results')
    df_res_load.to_excel(writer, sheet_name='Load Results')
    df_res_shunt.to_excel(writer, sheet_name='Shunt Results')
    df_res_sgen.to_excel(writer, sheet_name='Sgen Results')
    df_res_gen.to_excel(writer, sheet_name='Gen Results')
    df_cantidad_elementos.to_excel(writer, sheet_name='Excedentes operacionales', index=False)

### Apartir de este gráfico se puede destacar que el costo dual para las barras es de 100 + perdidas

In [5]:
df_res_bus['datetime'] = pd.to_datetime(df_res_bus['datetime'])

fig = go.Figure()

for bus_name in df_res_bus['bus_name'].unique():
    df_bus = df_res_bus[df_res_bus['bus_name'] == bus_name]
    
    fig.add_trace(
        go.Scatter(
            x=df_bus['datetime'], 
            y=df_bus['lam_p'], 
            name=f'lam_p - {bus_name}'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df_bus['datetime'], 
            y=df_bus['lam_q'], 
            name=f'lam_q - {bus_name}'
        )
    )

fig.update_layout(
    title="lam_p y lam_q para cada barra",
    xaxis_title="Fecha y hora",
    yaxis_title="Valor",
    xaxis_rangeslider_visible=True
)

fig.show()

df_res_cost['datetime'] = pd.to_datetime(df_res_cost['datetime'])

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_res_cost['datetime'], y=df_res_cost['Cost'], name="Costos")
)

fig.update_layout(
    title="Costos",
    xaxis_title="Fecha y hora",
    yaxis_title="$" ,
    xaxis_rangeslider_visible=False
)

fig.show()

path_output = 'Data/Output/Results.xlsx'
# Perdidas 

df_cantidad_elementos['datetime'] = pd.to_datetime(df_cantidad_elementos['datetime'])

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_cantidad_elementos['datetime'], y=df_cantidad_elementos['Perdidas de linea potencia activa (MW)'], name="Perdidas de linea potencia activa (MW)")
)

fig.update_layout(
    title="'Perdidas de potencia activa en las líneas de la red",
    xaxis_title="Fecha y hora",
    yaxis_title="MW" ,
    xaxis_rangeslider_visible=True
)

fig.show()

# Potencia suministrada por la red

df1 = pd.read_excel(path_output, sheet_name='External Grid Results')
df1['datetime'] = pd.to_datetime(df1['datetime'])

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df1['datetime'], y=df1['p_mw'], name='Potencia Activa (MW) - Red Externa')
)
fig.add_trace(
    go.Scatter(x=df1['datetime'], y=df1['q_mvar'], name='Potencia Reactiva (MVAR) - Red Externa')
)

df2 = pd.read_excel(path_output, sheet_name='Gen Results')

for gen_name in df2['gen_name'].unique():
    df_gen = df2[df2['gen_name'] == gen_name]

    if gen_name == 'gen 1':
        type = 'Diesel'
    elif gen_name == 'gen 2':
        type = 'Solar'

    fig.add_trace(
        go.Scatter(
            x=df_gen['datetime'], 
            y=df_gen['p_mw'], 
            mode='lines+markers',
            name=f'Potencia Activa (MW) - {gen_name} - {type}'
        )
    )
fig.update_layout(
    title="Potencia activa y reactiva suministrada por external grid",
    xaxis_title="Fecha y hora",
    yaxis_title="MW/MVAr",
    xaxis_rangeslider_visible=True
)

fig.show()


df = pd.read_excel(path_output, sheet_name='Load Results')


df['datetime'] = pd.to_datetime(df['datetime'])


fig = go.Figure()

cargas = df['load_name'].unique()


for carga in cargas:
    df_carga = df[df['load_name'] == carga]
    fig.add_trace(
        go.Scatter(
            x=df_carga['datetime'], 
            y=df_carga['p_mw'], 
            mode='lines',
            name=f"Carga {carga}"
        )
    )


fig.update_layout(
    title="Valor de las cargas en el tiempo",
    xaxis_title="Fecha y hora",
    yaxis_title="Potencia Activa (MW)",
    xaxis_rangeslider_visible=True
)


fig.show()



df_res_trafo = pd.read_excel(path_output, sheet_name='Transformer Results')

df_res_trafo['datetime'] = pd.to_datetime(df_res_trafo['datetime'])

fig = go.Figure()


transformadores = df_res_trafo['trafo_name'].unique()


for trafo in transformadores:
    df_trafo = df_res_trafo[df_res_trafo['trafo_name'] == trafo]
    fig.add_trace(
        go.Scatter(
            x=df_trafo['datetime'], 
            y=df_trafo['loading_percent'], 
            name=f"Carga transformador {trafo}"
        )
    )

fig.update_layout(
    title="Carga en los transformadores",
    xaxis_title="Fecha y hora", 
    yaxis_title="%",
    xaxis_rangeslider_visible=True
)

fig.show()

pp.plotting.plotly.pf_res_plotly(net, cmap="Jet", use_line_geodata=None, on_map=False, projection=None,
                  width_fig = 1200, height_fig = 900, line_width=2, bus_size=10,
                  climits_volt=(0.9, 1.1), climits_load=(0, 100), cpos_volt=1.0, cpos_load=1.1
                  )



# Costo promedio total para la operación de la red

Se puede notar que el costo es fijo, de 100$

In [6]:
df1 = pd.read_excel(path_output, sheet_name='External Grid Results')
df1['datetime'] = pd.to_datetime(df1['datetime'])

df_merged = pd.merge(df_res_cost, df1[['datetime', 'p_mw']], on='datetime', how='inner')

df_merged['costo_promedio'] = df_merged['Cost'] / df_merged['p_mw']

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_merged['datetime'], 
               y=df_merged['costo_promedio'], 
               name="Costo por MW")
)

fig.update_layout(
    title="Costo promedio para la external grid por MW",
    xaxis_title="Fecha y hora",
    yaxis_title="$/MW",
    xaxis_rangeslider_visible=False
)

print(df_merged[['datetime', 'Cost', 'p_mw', 'costo_promedio']].head())

fig.show()

             datetime       Cost      p_mw  costo_promedio
0 2025-03-03 00:00:00  48.118262  0.481183           100.0
1 2025-03-03 01:00:00  45.563239  0.455632           100.0
2 2025-03-03 02:00:00  43.773805  0.437738           100.0
3 2025-03-03 03:00:00  42.666667  0.426667           100.0
4 2025-03-03 04:00:00  42.031332  0.420313           100.0


# Costo promedio de las cargas

Aquí se consideran también las pérididas

In [7]:
df = pd.read_excel(path_output, sheet_name='Load Results')
df['datetime'] = pd.to_datetime(df['datetime'])

df_potencia_total = df.groupby('datetime')['p_mw'].sum().reset_index()

df_merged = pd.merge(df_res_cost, df_potencia_total, on='datetime', how='inner')

df_merged['costo_promedio'] = df_merged['Cost'] / df_merged['p_mw']

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_merged['datetime'], 
               y=df_merged['costo_promedio'], 
               name="Costo por MW consumido")
)

fig.update_layout(
    title="Costo promedio de operación por MW de carga",
    xaxis_title="Fecha y hora",
    yaxis_title="$/MW",
    xaxis_rangeslider_visible=True
)

print(df_merged[['datetime', 'Cost', 'p_mw', 'costo_promedio']].head())

fig.show()

             datetime       Cost      p_mw  costo_promedio
0 2025-03-03 00:00:00  48.118262  0.472791      101.774855
1 2025-03-03 01:00:00  45.563239  0.448867      101.507245
2 2025-03-03 02:00:00  43.773805  0.431968      101.335745
3 2025-03-03 03:00:00  42.666667  0.421454      101.236771
4 2025-03-03 04:00:00  42.031332  0.415400      101.182842


# Costo promedio de las pérdidas


In [8]:
df = pd.read_excel(path_output, sheet_name='Load Results')
df['datetime'] = pd.to_datetime(df['datetime'])

df_potencia_total = df.groupby('datetime')['p_mw'].sum().reset_index()

df_merged = pd.merge(df_res_cost, df_potencia_total, on='datetime', how='inner')

df_merged['costo_promedio'] = df_merged['Cost'] / df_merged['p_mw'] -100

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_merged['datetime'], 
               y=df_merged['costo_promedio'], 
               name="Costo por MW consumido")
)

fig.update_layout(
    title="Costo promedio pérdidas",
    xaxis_title="Fecha y hora",
    yaxis_title="$/MW",
    xaxis_rangeslider_visible=True
)

print(df_merged[['datetime', 'Cost', 'p_mw', 'costo_promedio']].head())

fig.show()

             datetime       Cost      p_mw  costo_promedio
0 2025-03-03 00:00:00  48.118262  0.472791        1.774855
1 2025-03-03 01:00:00  45.563239  0.448867        1.507245
2 2025-03-03 02:00:00  43.773805  0.431968        1.335745
3 2025-03-03 03:00:00  42.666667  0.421454        1.236771
4 2025-03-03 04:00:00  42.031332  0.415400        1.182842


# Prueba 2 : Cargas con beneficio de 100$
    - Las cargas ahora presentan un beneficio de 100$/1MW
    - Sin restricciones activas

In [9]:
# Definicion de la red
net = pn.create_cigre_network_lv()

net.switch.loc[1, 'closed'] = False
net.switch.loc[2, 'closed'] = False
net.switch.loc[0, 'closed'] = True  #Solo switch residencial activado

loads = [0, 1, 2, 3, 4, 5] #Cargas residenciales
no_loads = [ 6, 7, 8, 9, 10, 11, 12, 13, 14] #Cargas no residenciales

net.load.loc[loads, 'q_mvar'] = 0
net.load.loc[no_loads, 'in_service'] = False
net.load.loc[net.load['name'] == 'Load R1', 'p_mw'] *= 1.7

##Restricciones
for bus in net.bus.index:
    net.bus.at[bus, 'min_vm_pu'] = 0.8
    net.bus.at[bus, 'max_vm_pu'] = 1.2

for trafo in net.trafo.index:
    net.trafo.at[trafo, 'max_loading_percent'] = 200

for line in net.line.index:
    net.line.at[line, 'max_loading_percent'] = 200  # Considerar que la potencia nominal del transformador son 0.5 MVA


## Costos para la función objetivo
pp.create_pwl_cost(net, 0, 'ext_grid', [[0, 1.0, 100]])  # Un costo de 100$ para 1 MW

load = ['Load R11', 'Load R15', 'Load R16','Load R17',  'Load R18']

for load_name in load:
    load_index = net.load[net.load['name'] == load_name].index[0]
    pp.create_pwl_cost(net, load_index, 'load', [[0, 1.0, -100]])


In [10]:
df_res_bus = pd.DataFrame()
df_res_line = pd.DataFrame()
df_res_trafo = pd.DataFrame()
df_res_ext_grid = pd.DataFrame()
df_res_load = pd.DataFrame()
df_res_shunt = pd.DataFrame()
df_res_sgen = pd.DataFrame()
df_res_gen = pd.DataFrame()
df_cantidad_elementos = pd.DataFrame()

df_res_cost = pd.DataFrame()
df_cargas_restringidas = pd.DataFrame(columns=['timestamp', 'load_name', 'expected_p_mw', 'real_p_mw', 'restricted_p_mw'])

frames = []

for index, row in Demanda_alimentadores.iterrows():
    fecha_local = row['timestamp']
    
    for idx, load_row in net.load.iterrows():
        load_name = load_row['name']
        
        if load_name == 'Load R11':
            expected_p_mw = float(row['load1']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 4000
        
        elif load_name == 'Load R15':
            expected_p_mw = float(row['load10']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 12000    #15000
            net.load.at[idx, 'max_p_mw' ] = expected_p_mw / 12000   #15000
        
        elif load_name == 'Load R16':
            expected_p_mw = float(row['load15']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 5500
        
        elif load_name == 'Load R17':
            expected_p_mw = float(row['load20']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 21000
        
        elif load_name == 'Load R18':
            expected_p_mw = float(row['load25']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 21000

    pp.runopp(net,verbose=False) 
    
    net.res_bus['datetime'] = fecha_local
    net.res_line['datetime'] = fecha_local
    net.res_trafo['datetime'] = fecha_local
    net.res_ext_grid['datetime'] = fecha_local
    net.res_load['datetime'] = fecha_local
    net.res_shunt['datetime'] = fecha_local
    net.res_sgen['datetime'] = fecha_local
    net.res_gen['datetime'] = fecha_local

    df_res_cost_xd = pd.DataFrame({'Cost': [net.res_cost], 'datetime': [fecha_local]})

    net.res_gen['type'] = net.gen['type']

    net.res_bus['bus_name'] = net.bus['name']
    net.res_line['line_name'] = net.line['name']
    net.res_trafo['trafo_name'] = net.trafo['name']
    net.res_ext_grid['ext_grid_name'] = net.ext_grid['name']
    net.res_load['load_name'] = net.load['name']
    net.res_shunt['shunt_name'] = net.shunt['name']
    net.res_sgen['sgen_name'] = net.sgen['name']
    net.res_gen['gen_name'] = net.gen['name']

    net.res_sgen['bus'] = net.sgen['bus']
    net.res_gen['bus'] = net.gen['bus']
    net.res_load['type'] = net.load['type']

    ext_grid_buses = net.ext_grid['bus'].values
    ext_grid_bus_names = net.bus.loc[ext_grid_buses, 'name'].values

    net.res_ext_grid['bus_name'] = ext_grid_bus_names
    net.res_ext_grid['bus_index'] = ext_grid_buses

    load_buses = net.load['bus'].values
    load_bus_names = net.bus.loc[load_buses, 'name'].values
    net.res_load['bus_name'] = load_bus_names
    net.res_load['bus_index'] = load_buses
    from_bus_names = net.bus.loc[net.line['from_bus'], 'name'].values
    to_bus_names = net.bus.loc[net.line['to_bus'], 'name'].values

    net.res_line['from_bus_name'] = from_bus_names
    net.res_line['to_bus_name'] = to_bus_names
    net.res_line['from_bus'] = net.line['from_bus']
    net.res_line['to_bus'] = net.line['to_bus']

    line_active_losses = net.res_line.pl_mw.sum()
    line_reactive_losses = net.res_line.ql_mvar.sum()

    df_temp = pd.DataFrame({
        'Perdidas de linea potencia activa (MW)': [line_active_losses],
        'Perdidas de linea potencia reactiva (MVAR)': [line_reactive_losses],
        'datetime': [fecha_local]
    })
    df_res_cost = pd.concat([df_res_cost, df_res_cost_xd])
    df_res_bus = pd.concat([df_res_bus, net.res_bus])
    df_res_line = pd.concat([df_res_line, net.res_line])
    df_res_trafo = pd.concat([df_res_trafo, net.res_trafo])
    df_res_ext_grid = pd.concat([df_res_ext_grid, net.res_ext_grid])
    df_res_load = pd.concat([df_res_load, net.res_load])
    df_res_shunt = pd.concat([df_res_shunt, net.res_shunt])
    df_res_sgen = pd.concat([df_res_sgen, net.res_sgen])
    df_res_gen = pd.concat([df_res_gen, net.res_gen])
    df_cantidad_elementos = pd.concat([df_cantidad_elementos, df_temp])

with pd.ExcelWriter('Data/Output/Results.xlsx') as writer:
    df_res_cost.to_excel(writer, sheet_name='Cost')
    df_res_bus.to_excel(writer, sheet_name='Bus Results')
    df_res_line.to_excel(writer, sheet_name='Line Results')
    df_res_trafo.to_excel(writer, sheet_name='Transformer Results')
    df_res_ext_grid.to_excel(writer, sheet_name='External Grid Results')
    df_res_load.to_excel(writer, sheet_name='Load Results')
    df_res_shunt.to_excel(writer, sheet_name='Shunt Results')
    df_res_sgen.to_excel(writer, sheet_name='Sgen Results')
    df_res_gen.to_excel(writer, sheet_name='Gen Results')
    df_cantidad_elementos.to_excel(writer, sheet_name='Excedentes operacionales', index=False)

## Los resultados para este caso son similares al anterior

In [11]:
df_res_bus['datetime'] = pd.to_datetime(df_res_bus['datetime'])

fig = go.Figure()

for bus_name in df_res_bus['bus_name'].unique():
    df_bus = df_res_bus[df_res_bus['bus_name'] == bus_name]
    
    fig.add_trace(
        go.Scatter(
            x=df_bus['datetime'], 
            y=df_bus['lam_p'], 
            name=f'lam_p - {bus_name}'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df_bus['datetime'], 
            y=df_bus['lam_q'], 
            name=f'lam_q - {bus_name}'
        )
    )

fig.update_layout(
    title="lam_p y lam_q para cada barra",
    xaxis_title="Fecha y hora",
    yaxis_title="Valor",
    xaxis_rangeslider_visible=True
)

fig.show()

df_res_cost['datetime'] = pd.to_datetime(df_res_cost['datetime'])

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_res_cost['datetime'], y=df_res_cost['Cost'], name="Costos")
)

fig.update_layout(
    title="Costos",
    xaxis_title="Fecha y hora",
    yaxis_title="$" ,
    xaxis_rangeslider_visible=False
)

fig.show()

path_output = 'Data/Output/Results.xlsx'
# Perdidas 

df_cantidad_elementos['datetime'] = pd.to_datetime(df_cantidad_elementos['datetime'])

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_cantidad_elementos['datetime'], y=df_cantidad_elementos['Perdidas de linea potencia activa (MW)'], name="Perdidas de linea potencia activa (MW)")
)

fig.update_layout(
    title="'Perdidas de potencia activa en las líneas de la red",
    xaxis_title="Fecha y hora",
    yaxis_title="MW" ,
    xaxis_rangeslider_visible=True
)

fig.show()

# Potencia suministrada por la red

df1 = pd.read_excel(path_output, sheet_name='External Grid Results')
df1['datetime'] = pd.to_datetime(df1['datetime'])

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df1['datetime'], y=df1['p_mw'], name='Potencia Activa (MW) - Red Externa')
)
fig.add_trace(
    go.Scatter(x=df1['datetime'], y=df1['q_mvar'], name='Potencia Reactiva (MVAR) - Red Externa')
)

df2 = pd.read_excel(path_output, sheet_name='Gen Results')

for gen_name in df2['gen_name'].unique():
    df_gen = df2[df2['gen_name'] == gen_name]

    if gen_name == 'gen 1':
        type = 'Diesel'
    elif gen_name == 'gen 2':
        type = 'Solar'

    fig.add_trace(
        go.Scatter(
            x=df_gen['datetime'], 
            y=df_gen['p_mw'], 
            mode='lines+markers',
            name=f'Potencia Activa (MW) - {gen_name} - {type}'
        )
    )
fig.update_layout(
    title="Potencia activa y reactiva suministrada por external grid",
    xaxis_title="Fecha y hora",
    yaxis_title="MW/MVAr",
    xaxis_rangeslider_visible=True
)

fig.show()


df = pd.read_excel(path_output, sheet_name='Load Results')


df['datetime'] = pd.to_datetime(df['datetime'])


fig = go.Figure()

cargas = df['load_name'].unique()


for carga in cargas:
    df_carga = df[df['load_name'] == carga]
    fig.add_trace(
        go.Scatter(
            x=df_carga['datetime'], 
            y=df_carga['p_mw'], 
            mode='lines',
            name=f"Carga {carga}"
        )
    )


fig.update_layout(
    title="Valor de las cargas en el tiempo",
    xaxis_title="Fecha y hora",
    yaxis_title="Potencia Activa (MW)",
    xaxis_rangeslider_visible=True
)


fig.show()



df_res_trafo = pd.read_excel(path_output, sheet_name='Transformer Results')

df_res_trafo['datetime'] = pd.to_datetime(df_res_trafo['datetime'])

fig = go.Figure()


transformadores = df_res_trafo['trafo_name'].unique()


for trafo in transformadores:
    df_trafo = df_res_trafo[df_res_trafo['trafo_name'] == trafo]
    fig.add_trace(
        go.Scatter(
            x=df_trafo['datetime'], 
            y=df_trafo['loading_percent'], 
            name=f"Carga transformador {trafo}"
        )
    )

fig.update_layout(
    title="Carga en los transformadores",
    xaxis_title="Fecha y hora", 
    yaxis_title="%",
    xaxis_rangeslider_visible=True
)

fig.show()

pp.plotting.plotly.pf_res_plotly(net, cmap="Jet", use_line_geodata=None, on_map=False, projection=None,
                  width_fig = 1200, height_fig = 900, line_width=2, bus_size=10,
                  climits_volt=(0.9, 1.1), climits_load=(0, 100), cpos_volt=1.0, cpos_load=1.1
                  )



In [12]:
df1 = pd.read_excel(path_output, sheet_name='External Grid Results')
df1['datetime'] = pd.to_datetime(df1['datetime'])

df_merged = pd.merge(df_res_cost, df1[['datetime', 'p_mw']], on='datetime', how='inner')

df_merged['costo_promedio'] = df_merged['Cost'] / df_merged['p_mw']

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_merged['datetime'], 
               y=df_merged['costo_promedio'], 
               name="Costo por MW")
)

fig.update_layout(
    title="Costo promedio para la external grid por MW",
    xaxis_title="Fecha y hora",
    yaxis_title="$/MW",
    xaxis_rangeslider_visible=False
)

print(df_merged[['datetime', 'Cost', 'p_mw', 'costo_promedio']].head())

fig.show()

             datetime       Cost      p_mw  costo_promedio
0 2025-03-03 00:00:00  48.605339  0.486053           100.0
1 2025-03-03 01:00:00  45.983277  0.459833           100.0
2 2025-03-03 02:00:00  44.140362  0.441404           100.0
3 2025-03-03 03:00:00  42.994647  0.429946           100.0
4 2025-03-03 04:00:00  42.333832  0.423338           100.0


In [13]:
df = pd.read_excel(path_output, sheet_name='Load Results')
df['datetime'] = pd.to_datetime(df['datetime'])

df_potencia_total = df.groupby('datetime')['p_mw'].sum().reset_index()

df_merged = pd.merge(df_res_cost, df_potencia_total, on='datetime', how='inner')

df_merged['costo_promedio'] = df_merged['Cost'] / df_merged['p_mw']

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_merged['datetime'], 
               y=df_merged['costo_promedio'], 
               name="Costo por MW consumido")
)

fig.update_layout(
    title="Costo promedio para las cargas",
    xaxis_title="Fecha y hora",
    yaxis_title="$/MW",
    xaxis_rangeslider_visible=True
)

print(df_merged[['datetime', 'Cost', 'p_mw', 'costo_promedio']].head())

fig.show()

             datetime       Cost      p_mw  costo_promedio
0 2025-03-03 00:00:00  48.605339  0.477285      101.837063
1 2025-03-03 01:00:00  45.983277  0.452783      101.556904
2 2025-03-03 02:00:00  44.140362  0.435412      101.375986
3 2025-03-03 03:00:00  42.994647  0.424552      101.270657
4 2025-03-03 04:00:00  42.333832  0.418266      101.212776


In [14]:
df = pd.read_excel(path_output, sheet_name='Load Results')
df['datetime'] = pd.to_datetime(df['datetime'])

df_potencia_total = df.groupby('datetime')['p_mw'].sum().reset_index()

df_merged = pd.merge(df_res_cost, df_potencia_total, on='datetime', how='inner')

df_merged['costo_promedio'] = df_merged['Cost'] / df_merged['p_mw'] -100

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_merged['datetime'], 
               y=df_merged['costo_promedio'], 
               name="Costo por MW consumido")
)

fig.update_layout(
    title="Costo promedio para las pérdidas",
    xaxis_title="Fecha y hora",
    yaxis_title="$/MW",
    xaxis_rangeslider_visible=True
)

print(df_merged[['datetime', 'Cost', 'p_mw', 'costo_promedio']].head())

fig.show()

             datetime       Cost      p_mw  costo_promedio
0 2025-03-03 00:00:00  48.605339  0.477285        1.837063
1 2025-03-03 01:00:00  45.983277  0.452783        1.556904
2 2025-03-03 02:00:00  44.140362  0.435412        1.375986
3 2025-03-03 03:00:00  42.994647  0.424552        1.270657
4 2025-03-03 04:00:00  42.333832  0.418266        1.212776


## Prueba 3
    - Cargas fijas.
    - Se activan las restricciones de voltaje y capacidad del transformador (0.5 MVA).
    - Se agrega un generador de falla en la barra R15 con un costo de 1000.
    - Se aumentan las cargas gradualmente para activar las las restricciones.
    - Se quitan las pérdidas de líneas (para esté caso generan problemas de convergencia).

In [15]:
# Definicion de la red
net = pn.create_cigre_network_lv()

net.switch.loc[1, 'closed'] = False
net.switch.loc[2, 'closed'] = False
net.switch.loc[0, 'closed'] = True  #Solo switch residencial activado

loads = [0, 1, 2, 3, 4, 5] #Cargas residenciales
no_loads = [ 6, 7, 8, 9, 10, 11, 12, 13, 14] #Cargas no residenciales

net.load.loc[loads, 'q_mvar'] = 0
net.load.loc[no_loads, 'in_service'] = False
net.load.loc[net.load['name'] == 'Load R1', 'p_mw'] *= 1.7

##Restricciones
for bus in net.bus.index:
    net.bus.at[bus, 'min_vm_pu'] = 0.95
    net.bus.at[bus, 'max_vm_pu'] = 1.05

##Restricciones
for line in net.line.index:
    net.line.at[line, 'r_ohm_per_km'] = 0

for trafo in net.trafo.index:
    net.trafo.at[trafo, 'max_loading_percent'] = 100

for line in net.line.index:
    net.line.at[line, 'max_loading_percent'] = 200 # Considerar que la potencia nominal del transformador son 0.5 MVA


## Costos para la función objetivo
pp.create_pwl_cost(net, 0, 'ext_grid', [[0, 1000, 100]])  # Un costo de 100$ para 1 MW

load = ['Load R11', 'Load R15', 'Load R16','Load R17',  'Load R18']

for load_name in load:
    load_index = net.load[net.load['name'] == load_name].index[0]
    pp.create_pwl_cost(net, load_index, 'load', [[0, 1000, -100]])


gen = pp.create_sgen(net, bus=16, p_mw=0, name='SGen 1', controllable=True, max_q_mvar=0, min_q_mvar=0,  max_p_mw=1, min_p_mw=0)
## Costos para el generador - función objetivo
pp.create_pwl_cost(net, gen, 'sgen', [[0,1000,1000]])

6

In [16]:
df_res_bus = pd.DataFrame()
df_res_line = pd.DataFrame()
df_res_trafo = pd.DataFrame()
df_res_ext_grid = pd.DataFrame()
df_res_load = pd.DataFrame()
df_res_shunt = pd.DataFrame()
df_res_sgen = pd.DataFrame()
df_res_gen = pd.DataFrame()
df_cantidad_elementos = pd.DataFrame()

df_res_cost = pd.DataFrame()
df_cargas_restringidas = pd.DataFrame(columns=['timestamp', 'load_name', 'expected_p_mw', 'real_p_mw', 'restricted_p_mw'])

frames = []

for index, row in Demanda_alimentadores.iterrows():
    fecha_local = row['timestamp']
    print(fecha_local)
    for idx, load_row in net.load.iterrows():
        load_name = load_row['name']
        
        if load_name == 'Load R11':
            expected_p_mw = float(row['load1']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 4000
        
        elif load_name == 'Load R15':
            expected_p_mw = float(row['load10']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 15000     
        
        elif load_name == 'Load R16':
            expected_p_mw = float(row['load15']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 8500
        
        elif load_name == 'Load R17':
            expected_p_mw = float(row['load20']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 21000
        
        elif load_name == 'Load R18':
            expected_p_mw = float(row['load25']**1)
            net.load.at[idx, 'p_mw'] = expected_p_mw / 21000

    pp.runopp(net,verbose=False) 
    
    net.res_bus['datetime'] = fecha_local
    net.res_line['datetime'] = fecha_local
    net.res_trafo['datetime'] = fecha_local
    net.res_ext_grid['datetime'] = fecha_local
    net.res_load['datetime'] = fecha_local
    net.res_shunt['datetime'] = fecha_local
    net.res_sgen['datetime'] = fecha_local
    net.res_gen['datetime'] = fecha_local

    df_res_cost_xd = pd.DataFrame({'Cost': [net.res_cost], 'datetime': [fecha_local]})

    net.res_gen['type'] = net.gen['type']

    net.res_bus['bus_name'] = net.bus['name']
    net.res_line['line_name'] = net.line['name']
    net.res_trafo['trafo_name'] = net.trafo['name']
    net.res_ext_grid['ext_grid_name'] = net.ext_grid['name']
    net.res_load['load_name'] = net.load['name']
    net.res_shunt['shunt_name'] = net.shunt['name']
    net.res_sgen['sgen_name'] = net.sgen['name']
    net.res_gen['gen_name'] = net.gen['name']

    net.res_sgen['bus'] = net.sgen['bus']
    net.res_gen['bus'] = net.gen['bus']
    net.res_load['type'] = net.load['type']

    ext_grid_buses = net.ext_grid['bus'].values
    ext_grid_bus_names = net.bus.loc[ext_grid_buses, 'name'].values

    net.res_ext_grid['bus_name'] = ext_grid_bus_names
    net.res_ext_grid['bus_index'] = ext_grid_buses

    load_buses = net.load['bus'].values
    load_bus_names = net.bus.loc[load_buses, 'name'].values
    net.res_load['bus_name'] = load_bus_names
    net.res_load['bus_index'] = load_buses
    from_bus_names = net.bus.loc[net.line['from_bus'], 'name'].values
    to_bus_names = net.bus.loc[net.line['to_bus'], 'name'].values

    net.res_line['from_bus_name'] = from_bus_names
    net.res_line['to_bus_name'] = to_bus_names
    net.res_line['from_bus'] = net.line['from_bus']
    net.res_line['to_bus'] = net.line['to_bus']

    line_active_losses = net.res_line.pl_mw.sum()
    line_reactive_losses = net.res_line.ql_mvar.sum()

    df_temp = pd.DataFrame({
        'Perdidas de linea potencia activa (MW)': [line_active_losses],
        'Perdidas de linea potencia reactiva (MVAR)': [line_reactive_losses],
        'datetime': [fecha_local]
    })
    df_res_cost = pd.concat([df_res_cost, df_res_cost_xd])
    df_res_bus = pd.concat([df_res_bus, net.res_bus])
    df_res_line = pd.concat([df_res_line, net.res_line])
    df_res_trafo = pd.concat([df_res_trafo, net.res_trafo])
    df_res_ext_grid = pd.concat([df_res_ext_grid, net.res_ext_grid])
    df_res_load = pd.concat([df_res_load, net.res_load])
    df_res_shunt = pd.concat([df_res_shunt, net.res_shunt])
    df_res_sgen = pd.concat([df_res_sgen, net.res_sgen])
    df_res_gen = pd.concat([df_res_gen, net.res_gen])
    df_cantidad_elementos = pd.concat([df_cantidad_elementos, df_temp])

with pd.ExcelWriter('Data/Output/Results.xlsx') as writer:
    df_res_cost.to_excel(writer, sheet_name='Cost')
    df_res_bus.to_excel(writer, sheet_name='Bus Results')
    df_res_line.to_excel(writer, sheet_name='Line Results')
    df_res_trafo.to_excel(writer, sheet_name='Transformer Results')
    df_res_ext_grid.to_excel(writer, sheet_name='External Grid Results')
    df_res_load.to_excel(writer, sheet_name='Load Results')
    df_res_shunt.to_excel(writer, sheet_name='Shunt Results')
    df_res_sgen.to_excel(writer, sheet_name='Sgen Results')
    df_res_gen.to_excel(writer, sheet_name='Gen Results')
    df_cantidad_elementos.to_excel(writer, sheet_name='Excedentes operacionales', index=False)

2025-03-03 00:00:00
2025-03-03 01:00:00
2025-03-03 02:00:00
2025-03-03 03:00:00
2025-03-03 04:00:00
2025-03-03 05:00:00
2025-03-03 06:00:00
2025-03-03 07:00:00
2025-03-03 08:00:00
2025-03-03 09:00:00
2025-03-03 10:00:00
2025-03-03 11:00:00
2025-03-03 12:00:00
2025-03-03 13:00:00
2025-03-03 14:00:00
2025-03-03 15:00:00
2025-03-03 16:00:00
2025-03-03 17:00:00
2025-03-03 18:00:00
2025-03-03 19:00:00
2025-03-03 20:00:00
2025-03-03 21:00:00
2025-03-03 22:00:00
2025-03-03 23:00:00


## apartir de las 18:00 hasta las 21:00 se activa el generador de falla para mantener a la red dentro de las restricciones, lo que ocaciona que los duales en ciertas barras se disparen

In [17]:
df_res_bus['datetime'] = pd.to_datetime(df_res_bus['datetime'])

fig = go.Figure()

for bus_name in df_res_bus['bus_name'].unique():
    df_bus = df_res_bus[df_res_bus['bus_name'] == bus_name]
    
    fig.add_trace(
        go.Scatter(
            x=df_bus['datetime'], 
            y=df_bus['lam_p'], 
            name=f'lam_p - {bus_name}'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df_bus['datetime'], 
            y=df_bus['lam_q'], 
            name=f'lam_q - {bus_name}'
        )
    )

fig.update_layout(
    title="lam_p y lam_q para cada barra",
    xaxis_title="Fecha y hora",
    yaxis_title="Valor",
    xaxis_rangeslider_visible=True
)

fig.show()

df_res_cost['datetime'] = pd.to_datetime(df_res_cost['datetime'])

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_res_cost['datetime'], y=df_res_cost['Cost'], name="Costos")
)

fig.update_layout(
    title="Costos",
    xaxis_title="Fecha y hora",
    yaxis_title="$" ,
    xaxis_rangeslider_visible=False
)

fig.show()

path_output = 'Data/Output/Results.xlsx'
# Perdidas 

df_cantidad_elementos['datetime'] = pd.to_datetime(df_cantidad_elementos['datetime'])

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_cantidad_elementos['datetime'], y=df_cantidad_elementos['Perdidas de linea potencia activa (MW)'], name="Perdidas de linea potencia activa (MW)")
)

fig.update_layout(
    title="'Perdidas de potencia activa en las líneas de la red",
    xaxis_title="Fecha y hora",
    yaxis_title="MW" ,
    xaxis_rangeslider_visible=True
)

fig.show()

# Potencia suministrada por la red

df1 = pd.read_excel(path_output, sheet_name='External Grid Results')
df1['datetime'] = pd.to_datetime(df1['datetime'])

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df1['datetime'], y=df1['p_mw'], name='Potencia Activa (MW) - Red Externa')
)
fig.add_trace(
    go.Scatter(x=df1['datetime'], y=df1['q_mvar'], name='Potencia Reactiva (MVAR) - Red Externa')
)


fig.update_layout(
    title="Potencia activa y reactiva suministrada por external grid",
    xaxis_title="Fecha y hora",
    yaxis_title="MW/MVAr",
    xaxis_rangeslider_visible=True
)

fig.show()


df = pd.read_excel(path_output, sheet_name='Load Results')


df['datetime'] = pd.to_datetime(df['datetime'])


fig = go.Figure()

cargas = df['load_name'].unique()


for carga in cargas:
    df_carga = df[df['load_name'] == carga]
    fig.add_trace(
        go.Scatter(
            x=df_carga['datetime'], 
            y=df_carga['p_mw'], 
            mode='lines',
            name=f"Carga {carga}"
        )
    )


fig.update_layout(
    title="Valor de las cargas en el tiempo",
    xaxis_title="Fecha y hora",
    yaxis_title="Potencia Activa (MW)",
    xaxis_rangeslider_visible=True
)


fig.show()


df_res_sgen['datetime'] = pd.to_datetime(df_res_sgen['datetime'])

fig = go.Figure()

for gen_name in df_res_sgen['sgen_name'].unique():
    df_gen = df_res_sgen[df_res_sgen['sgen_name'] == gen_name]
    
    fig.add_trace(
        go.Scatter(
            x=df_gen['datetime'], 
            y=df_gen['p_mw'], 
            name=f'p_mw- {gen_name}'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df_gen['datetime'], 
            y=df_gen['q_mvar'], 
            name=f'q_mvar - {gen_name}'
        )
    )

fig.update_layout(
    title="generadores",
    xaxis_title="Fecha y hora",
    yaxis_title="Valor",
    xaxis_rangeslider_visible=True
)

fig.show()



df_res_trafo = pd.read_excel(path_output, sheet_name='Transformer Results')

df_res_trafo['datetime'] = pd.to_datetime(df_res_trafo['datetime'])

fig = go.Figure()


transformadores = df_res_trafo['trafo_name'].unique()


for trafo in transformadores:
    df_trafo = df_res_trafo[df_res_trafo['trafo_name'] == trafo]
    fig.add_trace(
        go.Scatter(
            x=df_trafo['datetime'], 
            y=df_trafo['loading_percent'], 
            name=f"Carga transformador {trafo}"
        )
    )

fig.update_layout(
    title="Carga en los transformadores",
    xaxis_title="Fecha y hora", 
    yaxis_title="%",
    xaxis_rangeslider_visible=True
)

fig.show()

pp.plotting.plotly.pf_res_plotly(net, cmap="Jet", use_line_geodata=None, on_map=False, projection=None,
                  width_fig = 1200, height_fig = 900, line_width=2, bus_size=10,
                  climits_volt=(0.9, 1.1), climits_load=(0, 100), cpos_volt=1.0, cpos_load=1.1
                  )



In [18]:
df1 = pd.read_excel(path_output, sheet_name='External Grid Results')
df1['datetime'] = pd.to_datetime(df1['datetime'])


df_merged = pd.merge(df_res_cost, df1[['datetime', 'p_mw']], on='datetime', how='inner')

df_merged['costo_promedio'] = df_merged['Cost'] / df_merged['p_mw']

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_merged['datetime'], 
               y=df_merged['costo_promedio'], 
               name="Costo por MW")
)

fig.update_layout(
    title="Costo Promedio de operación asociado a la external grid",
    xaxis_title="Fecha y hora",
    yaxis_title="$/MW",
    xaxis_rangeslider_visible=False
)

print(df_merged[['datetime', 'Cost', 'p_mw', 'costo_promedio']].head())

fig.show()

             datetime       Cost      p_mw  costo_promedio
0 2025-03-03 00:00:00  45.962585  0.459625      100.000226
1 2025-03-03 01:00:00  43.882697  0.438826      100.000125
2 2025-03-03 02:00:00  42.393491  0.423934      100.000123
3 2025-03-03 03:00:00  41.452633  0.414526      100.000065
4 2025-03-03 04:00:00  40.896538  0.408965      100.000116
